In [0]:
import re
import pickle
import numpy as np
import nltk

from nltk.corpus import stopwords
from nltk.corpus import reuters
from nltk.classify.scikitlearn import SklearnClassifier
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.multioutput import ClassifierChain
from sklearn.metrics import f1_score,precision_score,recall_score

# Mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).




*   To write into a pickle file - 



```

import pickle

data1 = {'a': [1, 2, 3],
         'b': ('string', 'Unicode string'),
         'c': None}

selfref_list = [1, 2, 3]

output = open('data.pkl', 'wb')

pickle.dump(data1, output)
pickle.dump(selfref_list, output)

output.close()

```

*   To read from a pickled file -



```
pkl_file = open('data.pkl', 'rb')

data1 = pickle.load(pkl_file)
print(data1)

data2 = pickle.load(pkl_file)
print(data2)

pkl_file.close()

```



*   How to print all the keys in the dictionary and their respective values?



```
mydic = {"key 1":"value 1","key b":"value b"}
for key in mydic:
  print ("the key name is " + key + " and its value is " + mydic[key])

mydic = {"key 1":"value 1","key b":"value b"}
for key, value in mydic.items():
  print ("the key name is " + key + " and its value is " + mydic[key])
]

```













*   The process of converting data to something a computer can understand is referred to as pre-processing. One of the major forms of pre-processing is to filter out useless data. 

*   In natural language processing, useless words (data), are referred to as stop words. What are Stop words?
A stop word is a commonly used word (such as “the”, “a”, “an”, “in”) that a search engine has been programmed to ignore.


In [0]:
nltk.download('stopwords')
nltk.download('punkt')

stopWords = stopwords.words('english')

fileName = ['electronics_train_data.data','electronics_and_car_train_data.data'
	,'politics_and_car_train_data.data','politics_and_travel_train_data.data'
	,'politics_train_data.data','car_train_data.data','travel_train_data.data','travel_and_electronics_train_data.data','travel_and_car_train_data.data']

labelsFile = [['electronics'],['electronics','car'],['politics','car'],['politics','travel'],['politics'],['car'],['travel'],['travel','electronics'],['travel','car']]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
def getDocument(tweetList):
	result = ''
	for tweet in tweetList:
		result = result + ' '
		result = result + tweet
	return result

In [0]:
def readPickleFiles():
  labelData = []
  contentData = []
  for index in range(len(fileName)):
    pkl_file = open('/content/gdrive/My Drive/Datasets/TwitterData/' + fileName[index],'rb')
    getDict = pickle.load(pkl_file)
    for user in getDict:
      aggregatedDoc = getDocument(getDict[user])
      contentData.append(aggregatedDoc)
      labelData.append(labelsFile[index])
  return labelData,contentData



*   re.compile(pattern) compiles a regular expression pattern into a regular expression object, which can be used for matching using its match() and search() methods.


*  We wish to use regular expressions to search for pattern and to do this we're going to use meta characters.



```
.       - Any Character Except New Line
\d      - Digit (0-9)
\D      - Not a Digit (0-9)
\w      - Word Character (a-z, A-Z, 0-9, _)
\W      - Not a Word Character
\s      - Whitespace (space, tab, newline)
\S      - Not Whitespace (space, tab, newline)

\b      - Word Boundary
\B      - Not a Word Boundary
^       - Beginning of a String
$       - End of a String

[]      - Matches Characters in brackets
[^ ]    - Matches Characters NOT in brackets
|       - Either Or
( )     - Group

Quantifiers:
*       - 0 or More
+       - 1 or More
?       - 0 or One
{3}     - Exact Number
{3,4}   - Range of Numbers (Minimum, Maximum)

#### Example ####
emails = 'CoreyMSchafer@gmail.com, corey.schafer@university.edu, corey-321-schafer@my-work.net'
pattern = re.compile(r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+')
matches = pattern.finditer(emails)

for match in matches:
    print(match)


```



In [0]:
def getOnlyWords(wordsList):
	pattern = re.compile('[a-zA-Z]+')
	result = []
	for word in wordsList:
		if pattern.match(word) != None:
		  result.append(word)
	return result


In [0]:
def getWordsWithoutStop(sentence):
	sentence = sentence.lower()
	words = word_tokenize(sentence)
	withoutStopWords = [word for word in words if not word in stopWords]
	return getOnlyWords(withoutStopWords)



*   A tokenizer divides text into a sequence of tokens. The tokenizer argument should be a function that takes a string as an input.



```
RUN THE BELOW COMMENTED CODE TO UNDERSTAND LOWER CASING LETTERS, TOKENIZING WORDS, REMOVING STOPWORDS, REMOVING EVERY WORD EXCEPT THOSE CONTAINING [a-zA-Z]+ 

def getWordsWithoutStop(sentences): 
  sentences = sentences.lower()
  #print(sentences)

  withoutStopWords = []
  words = word_tokenize(sentences)
  #print(words)

  withoutStopWord = [word for word in words if not word in stopWords]
  #print(withoutStopWord)
  
  pattern = re.compile('[a-zA-Z]+')
  result = []
  for word in withoutStopWord:
    if pattern.match(word) != None:
      result.append(word)
  #print(result)
  return result

trainContent = ['Within minutes, the pro-government television channels were calling it a huge ‘game-changer’ but the “ten-percent quota” gambit could mean only one thing: the 5555 cabal will go to any length – do anything, say anything, promise anything – to try to win the next Lok Sabha election.', 'Never mind that the proposed constitutional amendment would certainly run afoul of the Supreme Court, but the super-clever men who constitute the 5555 clique clearly hope that large chunks of the friendly media will help the prime minister re-write the ‘narrative’ around this 10% business.', 'The 5555 crowd may well be entitled to its conceits but the democratic, liberal, progressive and secular voices owe it to the country not to get taken in by this sleight of hand.']
vectorizer = TfidfVectorizer(tokenizer = getWordsWithoutStop)
vectorizedTrainData = vectorizer.fit_transform(trainContent)
vectorizer.get_feature_names()
vectorizedTrainData.shape
vectorizedTrainData.toarray()
```







In [0]:
label, Content = readPickleFiles()
trainContent, testContent, labelTrain, labelTest = train_test_split(Content, label, test_size=.2, random_state=1)


In [0]:
vectorizer = TfidfVectorizer(tokenizer = getWordsWithoutStop)
vectorizedTrainData = vectorizer.fit_transform(trainContent)
vectorizedTestData = vectorizer.transform(testContent)



*  Multiclass classification means a classification task with more than two classes; e.g., classify a set of images of fruits which may be oranges, apples, or pears. 
Multilabel classification assigns to each sample a set of target labels. A text might be about any of religion, politics, finance or education at the same time or none of these.



*   Classifier Chain - A chain C1, · · · , C|L| of binary classifiers is formed. Each classifier Cj
in the chain is responsible for learning and predicting the binary association of label
lj given the feature space, augmented by all prior binary relevance predictions
in the chain l1, · · · , lj−1. The classification process begins at C1 and propagates
along the chain: C1 determines Pr(l1|x) and every following classifier C2 · · · C|L|
predicts P r(lj |xi, l1, . . . , lj−1). 


In [0]:
multiLabelBinary = MultiLabelBinarizer()
trainBinaryLabel = multiLabelBinary.fit_transform(labelTrain)
testBinaryLabel = multiLabelBinary.transform(labelTest)

In [0]:
classifierSVM = OneVsRestClassifier(LinearSVC())
classifierSVM.fit(vectorizedTrainData, trainBinaryLabel)

OneVsRestClassifier(estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
          n_jobs=None)

In [0]:
predictionsTrain = classifierSVM.predict(vectorizedTrainData)

precisionTrain = precision_score(trainBinaryLabel, predictionsTrain, average = 'micro')
print ("Linear SVM multi-label Train micro,")
print (precisionTrain)

precisionTrain = precision_score(trainBinaryLabel, predictionsTrain, average = 'macro')
print ("Linear SVM multi-label Train macro,")
print (precisionTrain)

predictionsTest = classifierSVM.predict(vectorizedTestData)

precisionTest = precision_score(testBinaryLabel, predictionsTest, average = 'micro')
print ("Linear SVM multi-label Test micro,")
print (precisionTest)

precisionTest = precision_score(testBinaryLabel, predictionsTest, average = 'macro')
print ("Linear SVM multi-label Test macro,")
print (precisionTest)


Linear SVM multi-label Train micro,
0.998556894436828
Linear SVM multi-label Train macro,
0.9986854981779145
Linear SVM multi-label Test micro,
0.9766949152542372
Linear SVM multi-label Test macro,
0.9776718388111761


In [0]:
randomChainClassifier = [ClassifierChain(LinearSVC(), order='random', random_state=i) for i in range(20)]
for chain in randomChainClassifier:
	chain.fit(vectorizedTrainData,trainBinaryLabel)

predictionsTrain = np.array([chain.predict(vectorizedTrainData) for chain in randomChainClassifier])

predictionsTrain = predictionsTrain.mean(axis=0)
threshold = [.1,.2,.3,.4,.5]
maxThreshold = 0.0
accuracyMax = 0.0

for thres in threshold:
	curPredictionsTrain = (predictionsTrain >= thres)
	precisionTrain = precision_score(trainBinaryLabel, curPredictionsTrain, average = 'micro')
	print ("ClassifierChain multi-label Train micro,")
	print (precisionTrain)
	precisionTrain = precision_score(trainBinaryLabel, curPredictionsTrain, average = 'macro')
	print ("ClassifierChain multi-label Train macro,")
	print (precisionTrain)
	if accuracyMax < precisionTrain:
		accuracyMax = precisionTrain
		maxThreshold = thres

predictionsTest = np.array([chain.predict(vectorizedTestData) for chain in randomChainClassifier])
predictionsTest = predictionsTest.mean(axis=0)
predictionsTest = (predictionsTest >= maxThreshold)

precisionTest = precision_score(testBinaryLabel, predictionsTest, average = 'micro')
print ("ClassifierChain multi-label Test micro,")
print (precisionTest)

precisionTest = precision_score(testBinaryLabel, predictionsTest, average = 'macro')
print ("ClassifierChain multi-label Test macro,")
print (precisionTest)

ClassifierChain multi-label Train micro,
0.9982338523644751
ClassifierChain multi-label Train macro,
0.998363272570492
ClassifierChain multi-label Train micro,
0.9983418045492232
ClassifierChain multi-label Train macro,
0.9984416667591766
ClassifierChain multi-label Train micro,
0.9984137284591535
ClassifierChain multi-label Train macro,
0.9985089431608121
ClassifierChain multi-label Train micro,
0.9984496683011249
ClassifierChain multi-label Train macro,
0.9985838039417672
ClassifierChain multi-label Train micro,
0.9984852856318522
ClassifierChain multi-label Train macro,
0.9986184804420662
ClassifierChain multi-label Test micro,
0.9694485842026825
ClassifierChain multi-label Test macro,
0.9700424010247494


In [0]:
testSet = ['A great blend of handheld comfort and a big, gorgeous OLED screen. Rear telephoto camera outshoots the 8 Plus in low light, and the front camera snaps impressive portrait selfies. Face ID generally works fine.'
,'Standard equipment for all Fit models includes a backup camera, automatic headlights, Bluetooth, LED taillights and a center console with an armrest.'
,'The term political statement is used to refer to any act or non-verbal form of communication that is intended to influence a decision to be made for or by a political party.'
,'The democrats are not happy about the trip to australia','I bought a bunch of automatic transmissions when Im travelling to to another country','I will be attending a birthday party today']

vectorizedTestData = vectorizer.transform(testSet)
predictionsTest = classifierSVM.predict(vectorizedTestData)
print ("Linear SVM multi-label manual input,")
print (multiLabelBinary.inverse_transform(predictionsTest))

predictionsTest = np.array([chain.predict(vectorizedTestData) for chain in randomChainClassifier])
predictionsTest = predictionsTest.mean(axis=0)
predictionsTest = (predictionsTest >= maxThreshold)

print ("ClassifierChain multi-label manual input,")
print (multiLabelBinary.inverse_transform(predictionsTest))


Linear SVM multi-label manual input,
[('electronics',), ('electronics',), ('politics',), ('travel',), ('car',), ('politics',)]
ClassifierChain multi-label manual input,
[('electronics',), ('electronics',), ('politics',), ('travel',), ('car', 'travel'), ('politics',)]
